In [30]:
import pandas as pd
import numpy as np

In [36]:
df = pd.read_csv('data/weather_data.csv')

In [37]:
df.head()

,time,temperature_2m,precipitation,relativehumidity_2m,windspeed_10m,winddirection_10m,surface_pressure,cloudcover,city
0,2026-01-17 00:00:00,7.4,0.1,91,9.7,129,1008.9,100,Edinburgh
1,2026-01-17 01:00:00,7.7,0.1,91,8.9,133,1009.0,100,Edinburgh
2,2026-01-17 02:00:00,7.9,0.1,91,7.9,133,1009.3,100,Edinburgh
3,2026-01-17 03:00:00,7.8,0.0,91,8.4,133,1008.7,100,Edinburgh
4,2026-01-17 04:00:00,7.8,0.0,91,7.4,133,1009.6,100,Edinburgh


In [38]:
df = df.rename(columns={'temperature_2m':'temperature','relativehumidity_2m': 'relative_humidity','windspeed_10m':'wind_speed','winddirection_10m':'wind_direction'})

In [39]:
df.shape

(174720, 9)

In [40]:
df.isna().sum()

time                 0
temperature          0
precipitation        0
relative_humidity    0
wind_speed           0
wind_direction       0
surface_pressure     0
cloudcover           0
city                 0
dtype: int64

In [41]:
df.duplicated().sum()

np.int64(0)

In [42]:
df.describe()

,temperature,precipitation,relative_humidity,wind_speed,wind_direction,surface_pressure,cloudcover
count,174720.000000,174720.000000,174720.000000,174720.000000,174720.000000,174720.000000,174720.000000
mean,8.273214,0.100595,87.142857,10.002976,130.404762,1003.048214,93.714286
std,1.188363,0.247968,5.791313,4.015094,31.769206,11.431886,10.231367
min,4.600000,0.000000,70.000000,2.700000,59.000000,986.500000,51.000000
25%,7.500000,0.000000,84.000000,6.800000,113.750000,989.675000,92.750000
50%,8.300000,0.000000,88.000000,9.050000,133.500000,1008.800000,98.000000
75%,8.900000,0.100000,91.000000,13.250000,151.250000,1012.825000,100.000000
max,10.900000,1.200000,96.000000,19.400000,193.000000,1017.800000,100.000000


In [28]:
df['temp_1h'] = df['temperature'].shift(1)  
df['temp_2h'] = df['temperature'].shift(2)  

In [ ]:
from sklearn.preprocessing

In [29]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso